# BP Tangguh Expansion Project (TEP) - Data Pipeline & EDA
## EPCI Contract Analysis (2016-2024)

**Objective:** Extract, clean, normalize, and analyze TEP project data from 3 Excel sources into 13 relational database tables for MySQL import.

**Data Sources:**
1. `Contract Value Overview and Timeline_15-Jul-24.xlsx` - Monthly time-series (FGRS, LOGI, POB, Subcontractors)
2. `TEP Contract Evolution.xlsx` - Amendment-level cost breakdown (Original → AMD-5)
3. `Cost Breakdown Structure.xlsx` - PAMF claim hierarchy (3,967 claims / $1.77B)

**Target:** 6 Master Tables (`tb_m_*`) + 7 Transaction Tables (`tb_t_*`) → MySQL

In [3]:
pip install pandas numpy matplotlib seaborn openpyxl

Defaulting to user installation because normal site-packages is not writeable




[notice] A new release of pip available: 22.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
print("hello world")

hello world


In [5]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.utils import get_column_letter
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

# Paths
RAW_DATA_DIR = r'D:\BP\raw_data'
OUTPUT_DIR = r'D:\BP\data_cleansing'

FILE_TIMELINE = os.path.join(RAW_DATA_DIR, 'Contract Value Overview and Timeline_15-Jul-24.xlsx')
FILE_EVOLUTION = os.path.join(RAW_DATA_DIR, 'TEP Contract Evolution.xlsx')
FILE_CBS = os.path.join(RAW_DATA_DIR, 'Cost Breakdown Structure.xlsx')

# Plot style
sns.set_theme(style='whitegrid', palette='deep')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['figure.dpi'] = 100

PROJECT_ID = 1
print('Setup complete. Output directory:', OUTPUT_DIR)

C:\Users\Stevia Putri\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Stevia Putri\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


Setup complete. Output directory: D:\BP\data_cleansing


---
## 1. RAW DATA EXTRACTION & STRUCTURE EXPLORATION
### 1.1 Load Excel Workbooks

In [6]:
wb_timeline = openpyxl.load_workbook(FILE_TIMELINE, data_only=True)
wb_evolution = openpyxl.load_workbook(FILE_EVOLUTION, data_only=True)
wb_cbs = openpyxl.load_workbook(FILE_CBS, data_only=True)

print("=" * 70)
print("FILE 1: Contract Value Overview and Timeline")
print(f"  Sheets: {wb_timeline.sheetnames}")
for sn in wb_timeline.sheetnames:
    ws = wb_timeline[sn]
    print(f"    '{sn}': {ws.max_row} rows x {ws.max_column} cols")

print("\nFILE 2: TEP Contract Evolution")
print(f"  Sheets: {wb_evolution.sheetnames}")
for sn in wb_evolution.sheetnames:
    ws = wb_evolution[sn]
    print(f"    '{sn}': {ws.max_row} rows x {ws.max_column} cols")

print("\nFILE 3: Cost Breakdown Structure")
print(f"  Sheets: {wb_cbs.sheetnames}")
for sn in wb_cbs.sheetnames:
    ws = wb_cbs[sn]
    print(f"    '{sn}': {ws.max_row} rows x {ws.max_column} cols")

FILE 1: Contract Value Overview and Timeline
  Sheets: ['Overall (with%prog)', 'Overall']
    'Overall (with%prog)': 193 rows x 89 cols
    'Overall': 53 rows x 89 cols

FILE 2: TEP Contract Evolution
  Sheets: ['Contract (2)', 'Contract']
    'Contract (2)': 47 rows x 9 cols
    'Contract': 47 rows x 9 cols

FILE 3: Cost Breakdown Structure
  Sheets: ['List PAMF']
    'List PAMF': 98 rows x 3 cols


In [7]:
# Peek at first rows of each file
print("=== TEP Contract Evolution - First 10 rows ===")
ws_evo = wb_evolution[wb_evolution.sheetnames[0]]
for row in range(1, min(12, ws_evo.max_row + 1)):
    vals = [ws_evo.cell(row=row, column=c).value for c in range(1, min(10, ws_evo.max_column + 1))]
    print(f"  Row {row}: {vals}")

print("\n=== Cost Breakdown Structure - First 10 rows ===")
ws_cbs_peek = wb_cbs[wb_cbs.sheetnames[0]]
for row in range(1, min(12, ws_cbs_peek.max_row + 1)):
    cell = ws_cbs_peek.cell(row=row, column=1)
    indent = cell.alignment.indent if cell.alignment and cell.alignment.indent else 0
    vals = [cell.value, ws_cbs_peek.cell(row=row, column=2).value, ws_cbs_peek.cell(row=row, column=3).value]
    print(f"  Row {row} (indent={indent}): {vals}")

print("\n=== Timeline - Row 1 (years) ===")
ws_main = wb_timeline[wb_timeline.sheetnames[0]]
year_row = [ws_main.cell(row=1, column=c).value for c in range(1, min(90, ws_main.max_column + 1))]
print(f"  {[v for v in year_row if v is not None]}")
print("\n=== Timeline - Row 3 (months) ===")
month_row = [ws_main.cell(row=3, column=c).value for c in range(1, min(90, ws_main.max_column + 1))]
print(f"  {[v for v in month_row if v is not None]}")
print("\n=== Timeline - Column A+B labels (rows 1-60) ===")
for row in range(1, min(61, ws_main.max_row + 1)):
    a = ws_main.cell(row=row, column=1).value
    b = ws_main.cell(row=row, column=2).value
    c_val = ws_main.cell(row=row, column=3).value
    if a or b or c_val:
        print(f"  Row {row}: A={a}, B={b}, C={c_val}")

=== TEP Contract Evolution - First 10 rows ===
  Row 1: [None, None, None, None, None, None, None, None, None]
  Row 2: [None, 'Total CONTRACT PRICE', None, 'Amount in USD', None, None, None, None, None]
  Row 3: [None, 'No', 'DESCRIPTION', 'Original Contract', 'Amendment 1', 'Amendment 2', 'Amendment 3', 'Amendment 4', 'Amendment 5']
  Row 4: [None, 'A', 'EPC LUMP SUM', 2396417606, 2330894610, 2666580182, 2643421716, 2643421716, 2643421716]
  Row 5: [None, 1, 'PROJECT MANAGEMENT', 157861778, 157861778, 179224169, 174224492, 174224492, 174224492]
  Row 6: [None, 2, 'ENGINEERING', 149829449, 149829449, 223418208, 222446401, 222446401, 222446401]
  Row 7: [None, 3, 'PROCUREMENT', 606018791, 606018791, 637862285, 637862285, 637862285, 637862285]
  Row 8: [None, 3.1, 'Procurement of Equipment', None, None, 315071544, 311065225, 311065225, 311065225]
  Row 9: [None, 3.2, 'Procurement of Bulk Material', None, None, 314408323, 326797060, 326797060, 326797060]
  Row 10: [None, 3.3, 'Capital Sp

### 1.2 Build Year-Month Column Mapping for Timeline File
The timeline uses merged year headers (Row 1) and month numbers (Row 3). Years 2017/2018 are aggregated as "1-12", 2024 is quarterly.

In [8]:
ws_main = wb_timeline[wb_timeline.sheetnames[0]]

def build_column_date_map(ws):
    """Map column indices to (year, month) tuples from merged header layout."""
    # Forward-fill years from row 1
    year_map = {}
    current_year = None
    for col in range(1, ws.max_column + 1):
        v = ws.cell(row=1, column=col).value
        if v is not None:
            try:
                current_year = int(v)
            except (ValueError, TypeError):
                pass
        if current_year:
            year_map[col] = current_year

    # Read months from row 3
    col_date_map = {}
    for col in range(1, ws.max_column + 1):
        month_val = ws.cell(row=3, column=col).value
        year = year_map.get(col)
        if month_val is None or year is None:
            continue
        if isinstance(month_val, (int, float)):
            col_date_map[col] = (year, int(month_val))
        elif isinstance(month_val, str):
            # Aggregated columns like '1 - 12', '1 - 3' etc
            col_date_map[col] = (year, month_val)
    return col_date_map

col_date_map = build_column_date_map(ws_main)

# Separate monthly vs aggregated
monthly_cols = {c: (y, m) for c, (y, m) in col_date_map.items() if isinstance(m, int)}
agg_cols = {c: (y, m) for c, (y, m) in col_date_map.items() if isinstance(m, str)}

print(f"Total column mappings: {len(col_date_map)}")
print(f"Monthly columns (usable): {len(monthly_cols)}")
print(f"Aggregated columns (skipped): {len(agg_cols)}")
if monthly_cols:
    print(f"Date range: {min(monthly_cols.values())} to {max(monthly_cols.values())}")
print(f"\nAggregated columns: {agg_cols}")

Total column mappings: 73
Monthly columns (usable): 67
Aggregated columns (skipped): 6
Date range: (2016, 6) to (2023, 12)

Aggregated columns: {11: (2017, '1 - 12'), 14: (2018, '1 - 12'), 77: (2024, '1 - 3'), 79: (2024, '4 - 6'), 81: (2024, '7 - 9'), 83: (2024, '10-12')}


In [9]:
def extract_time_series(ws, row_num, col_date_map, monthly_only=True):
    """Extract numeric values from a row using column-date mapping."""
    data = []
    for col, (year, month) in col_date_map.items():
        if monthly_only and not isinstance(month, int):
            continue
        val = ws.cell(row=row_num, column=col).value
        if val is not None and isinstance(val, (int, float)):
            data.append({'year': int(year), 'month': int(month) if isinstance(month, int) else None, 'value': float(val)})
    return data

print("Helper function extract_time_series() defined.")

Helper function extract_time_series() defined.


### 1.3 Extract TEP Contract Evolution Data

In [10]:
ws_evo = wb_evolution[wb_evolution.sheetnames[0]]

# Detect column layout
print("Column headers:")
for col in range(1, ws_evo.max_column + 1):
    h1 = ws_evo.cell(row=1, column=col).value
    h2 = ws_evo.cell(row=2, column=col).value
    h3 = ws_evo.cell(row=3, column=col).value
    print(f"  Col {col} ({get_column_letter(col)}): row1={h1}, row2={h2}, row3={h3}")

Column headers:
  Col 1 (A): row1=None, row2=None, row3=None
  Col 2 (B): row1=None, row2=Total CONTRACT PRICE, row3=No
  Col 3 (C): row1=None, row2=None, row3=DESCRIPTION
  Col 4 (D): row1=None, row2=Amount in USD, row3=Original Contract
  Col 5 (E): row1=None, row2=None, row3=Amendment 1
  Col 6 (F): row1=None, row2=None, row3=Amendment 2
  Col 7 (G): row1=None, row2=None, row3=Amendment 3
  Col 8 (H): row1=None, row2=None, row3=Amendment 4
  Col 9 (I): row1=None, row2=None, row3=Amendment 5


In [11]:
# Extract all data rows - adjust column mapping based on peek above
evolution_data = []
for row in range(1, ws_evo.max_row + 1):
    row_vals = {}
    for col in range(1, ws_evo.max_column + 1):
        row_vals[col] = ws_evo.cell(row=row, column=col).value
    evolution_data.append(row_vals)

df_evo_raw = pd.DataFrame(evolution_data)
print(f"Raw evolution data: {df_evo_raw.shape}")
df_evo_raw.head(50)

Raw evolution data: (47, 9)


,1,2,3,4,5,6,7,8,9
0,None,None,None,None,None,None,None,None,None
1,None,Total CONTRACT PRICE,None,Amount in USD,None,None,None,None,None
2,None,No,DESCRIPTION,Original Contract,Amendment 1,Amendment 2,Amendment 3,Amendment 4,Amendment 5
3,None,A,EPC LUMP SUM,2396417606,2330894610,2666580182,2643421716,2643421716,2643421716
4,None,1,PROJECT MANAGEMENT,157861778,157861778,179224169,174224492,174224492,174224492
5,None,2,ENGINEERING,149829449,149829449,223418208,222446401,222446401,222446401
6,None,3,PROCUREMENT,606018791,606018791,637862285,637862285,637862285,637862285
7,None,3.1,Procurement of Equipment,None,None,315071544,311065225,311065225,311065225
8,None,3.2,Procurement of Bulk Material,None,None,314408323,326797060,326797060,326797060
9,None,3.3,Capital Spare and Construction Spare,None,None,8382418,None,None,None


In [12]:
# Structure the evolution data properly
# Identify the columns: typically B=row_no, C=description, D-I = Original through AMD-5
# Find the header row and data columns by checking content
header_row = None
for i, row_data in enumerate(evolution_data):
    vals = list(row_data.values())
    vals_str = [str(v).upper() if v else '' for v in vals]
    if any('ORIGINAL' in v or 'CONTRACT' in v for v in vals_str):
        print(f"Potential header at index {i}: {vals}")
        if header_row is None:
            header_row = i

# Build structured dataframe from evolution
# Try standard layout: col 2=number, col 3=description, cols 4-9=amendments
evo_structured = []
for row in range(1, ws_evo.max_row + 1):
    row_no = ws_evo.cell(row=row, column=2).value
    desc = ws_evo.cell(row=row, column=3).value
    if desc is None:
        continue
    vals = {}
    vals['row_no'] = row_no
    vals['description'] = str(desc).strip()
    vals['excel_row'] = row
    # Try columns 4-9 for amendment values
    for ci, amd_name in enumerate(['original_contract', 'amd_1', 'amd_2', 'amd_3', 'amd_4', 'amd_5']):
        v = ws_evo.cell(row=row, column=4 + ci).value
        vals[amd_name] = v
    evo_structured.append(vals)

df_evolution = pd.DataFrame(evo_structured)
print(f"Structured evolution: {df_evolution.shape}")
print("\nDescriptions found:")
for _, r in df_evolution.iterrows():
    print(f"  [{r['row_no']}] {r['description']}: orig={r['original_contract']}")

Potential header at index 1: [None, 'Total CONTRACT PRICE', None, 'Amount in USD', None, None, None, None, None]
Potential header at index 2: [None, 'No', 'DESCRIPTION', 'Original Contract', 'Amendment 1', 'Amendment 2', 'Amendment 3', 'Amendment 4', 'Amendment 5']
Potential header at index 18: [None, 9, 'NOVATED CONTRACTS', 287133100, 221610104, 221610104, 221610104, 221610104, 221610104]
Potential header at index 19: [None, 9.1, 'APCI Contract', 37133100, 39111743, 39111743, 39111743, 39111743, 39111743]
Potential header at index 20: [None, 9.2, 'GE Contract', 250000000, 182498361, 182498361, 182498361, 182498361, 182498361]
Potential header at index 46: [None, None, 'Revised CONTRACT PRICE', 2432775726, 2367252730, 3140999999, 3628379837, 4264450678, 5047871388]
Structured evolution: (45, 9)

Descriptions found:
  [No] DESCRIPTION: orig=Original Contract
  [A] EPC LUMP SUM: orig=2396417606
  [1] PROJECT MANAGEMENT: orig=157861778
  [2] ENGINEERING: orig=149829449
  [3] PROCUREMENT: 

### 1.4 Extract Cost Breakdown Structure (PAMF Claims)

In [13]:
ws_cbs_data = wb_cbs[wb_cbs.sheetnames[0]]

pamf_data = []
current_l0 = None
current_l1 = None

for row in range(1, ws_cbs_data.max_row + 1):
    cell = ws_cbs_data.cell(row=row, column=1)
    label = cell.value
    indent = 0
    if cell.alignment and cell.alignment.indent:
        indent = int(cell.alignment.indent)
    count_val = ws_cbs_data.cell(row=row, column=2).value
    amount_val = ws_cbs_data.cell(row=row, column=3).value

    if label is None:
        continue
    label_str = str(label).strip()
    if label_str in ('', 'Grand Total', 'Row Labels', 'Count of PAMF', 'Sum of PAMF Claim Amount'):
        # Check if this is a header row
        if label_str == 'Grand Total' and count_val is not None:
            pamf_data.append({
                'discipline': 'TOTAL', 'category': None, 'subcategory': None,
                'label': label_str, 'level': -1,
                'claim_count': count_val, 'claim_amount_usd': amount_val,
                'excel_row': row
            })
        continue

    if indent == 0:
        current_l0 = label_str
        current_l1 = None
    elif indent == 1:
        current_l1 = label_str

    pamf_data.append({
        'discipline': current_l0 if indent >= 0 else None,
        'category': current_l1 if indent >= 1 else label_str if indent == 0 else None,
        'subcategory': label_str if indent == 2 else None,
        'label': label_str,
        'level': indent,
        'claim_count': count_val,
        'claim_amount_usd': amount_val,
        'excel_row': row
    })

df_pamf_raw = pd.DataFrame(pamf_data)
print(f"PAMF raw entries: {len(df_pamf_raw)}")
print(f"\nBy level:")
print(df_pamf_raw.groupby('level')[['claim_count', 'claim_amount_usd']].sum())
print(f"\nDisciplines (level 0):")
print(df_pamf_raw[df_pamf_raw['level'] == 0][['label', 'claim_count', 'claim_amount_usd']])

PAMF raw entries: 95

By level:
       claim_count  claim_amount_usd
level                               
-1            3967      1.771898e+09
 0            3967      1.771898e+09
 1            3967      1.771898e+09
 2            3967      1.771898e+09

Disciplines (level 0):
       label  claim_count  claim_amount_usd
0      covid           45      9.011710e+07
4   logistic          342      1.569896e+08
17       PMT          113      6.693467e+07
40       SMT         3467      1.457856e+09


### 1.5 Extract Time-Series Data from Timeline
Identify the correct rows for FGRS, LOGI, POB, and Subcontractor data.

In [14]:
# Scan row labels in column A, B, C to identify data rows
print("=== Timeline Row Labels (scanning cols A-C) ===")
row_labels = {}
for row in range(1, ws_main.max_row + 1):
    a = ws_main.cell(row=row, column=1).value
    b = ws_main.cell(row=row, column=2).value
    c = ws_main.cell(row=row, column=3).value
    label = a or b or c
    if label:
        label_str = str(label).strip()
        row_labels[row] = label_str
        # Check if this row has numeric data
        sample_vals = []
        for col in list(monthly_cols.keys())[:5]:
            v = ws_main.cell(row=row, column=col).value
            if v is not None and isinstance(v, (int, float)):
                sample_vals.append(v)
        has_data = "DATA" if sample_vals else "text"
        print(f"  Row {row:>3}: [{has_data:>4}] {label_str[:80]}  sample={sample_vals[:3]}")

=== Timeline Row Labels (scanning cols A-C) ===
  Row   5: [text] EPCI Contract
Key Dates  sample=[]
  Row  15: [text] CONTRACT VALUE 
TRANSFORMATION  sample=[]
  Row  18: [text] AMD-2  sample=[]
  Row  22: [text] AMD-3  sample=[]
  Row  27: [text] AMD-4  sample=[]
  Row  32: [text] AMD-5  sample=[]
  Row  37: [text] Post 
AMD-5  sample=[]
  Row  43: [text] FGRS RCE ACTUAL COST (MUSD)  sample=[]
  Row  44: [text] AMD-3 Est.Period  sample=[]
  Row  45: [text] AMD-4 Est.Period  sample=[]
  Row  46: [text] AMD-5 Est.Period  sample=[]
  Row  47: [text] Post AMD-5  sample=[]
  Row  49: [text] LOGI RCE ACTUAL COST (MUSD)  sample=[]
  Row  50: [text] AMD-3 Est.Period  sample=[]
  Row  51: [text] AMD-4 Est.Period  sample=[]
  Row  52: [text] AMD-5 Est.Period  sample=[]
  Row  53: [text] Post AMD-5  sample=[]
  Row  55: [text] POB (CSTS+SubCon)  sample=[]
  Row  58: [text] Workers at Site Isolation Facility  sample=[]
  Row  60: [text] KEY HIGHLIGHTS  sample=[]
  Row  62: [text] PIPING ERECTION

In [15]:
# Based on the scan above, identify key rows
# These may need adjustment based on actual file structure
# Let's search for specific keywords
fgrs_rows = [r for r, l in row_labels.items() if 'FGRS' in l.upper() or 'RCE' in l.upper()]
logi_rows = [r for r, l in row_labels.items() if 'LOGI' in l.upper()]
pob_rows = [r for r, l in row_labels.items() if 'POB' in l.upper()]
isolation_rows = [r for r, l in row_labels.items() if 'ISOLATION' in l.upper() or 'ISOL' in l.upper()]
meindo_rows = [r for r, l in row_labels.items() if 'MEINDO' in l.upper()]
penta_rows = [r for r, l in row_labels.items() if 'PENTA' in l.upper()]
daewoo_rows = [r for r, l in row_labels.items() if 'DAEWOO' in l.upper()]

print(f"FGRS-related rows: {fgrs_rows}")
print(f"LOGI-related rows: {logi_rows}")
print(f"POB-related rows: {pob_rows}")
print(f"Isolation rows: {isolation_rows}")
print(f"Meindo rows: {meindo_rows}")
print(f"Penta rows: {penta_rows}")
print(f"Daewoo rows: {daewoo_rows}")

# For each identified section, scan nearby rows for data
def find_data_rows(ws, start_row, num_rows=15):
    """Scan rows near a label to find which have numeric data."""
    results = []
    for r in range(max(1, start_row - 2), min(ws.max_row, start_row + num_rows)):
        label_a = ws.cell(row=r, column=1).value
        label_b = ws.cell(row=r, column=2).value
        label_c = ws.cell(row=r, column=3).value
        label = str(label_a or label_b or label_c or '').strip()
        data_count = 0
        for col in monthly_cols:
            v = ws.cell(row=r, column=col).value
            if v is not None and isinstance(v, (int, float)):
                data_count += 1
        if data_count > 0:
            results.append((r, label, data_count))
    return results

# Find data rows near each section
if meindo_rows:
    print("\n=== Meindo section ===")
    for r, l, c in find_data_rows(ws_main, meindo_rows[0]):
        print(f"  Row {r}: '{l}' ({c} data points)")

if penta_rows:
    print("\n=== Penta section ===")
    for r, l, c in find_data_rows(ws_main, penta_rows[0]):
        print(f"  Row {r}: '{l}' ({c} data points)")

if daewoo_rows:
    print("\n=== Daewoo section ===")
    for r, l, c in find_data_rows(ws_main, daewoo_rows[0]):
        print(f"  Row {r}: '{l}' ({c} data points)")

FGRS-related rows: [43, 49]
LOGI-related rows: [49]
POB-related rows: [55, 67, 81, 130, 131]
Isolation rows: [58]
Meindo rows: [63]
Penta rows: [77]
Daewoo rows: [126]

=== Meindo section ===
  Row 64: 'Achieved Mhrs' (51 data points)
  Row 65: 'Invoice Value (MUSD)' (34 data points)
  Row 66: 'Actual/VOWD' (36 data points)
  Row 67: 'POB' (47 data points)
  Row 68: 'Plan Cumm.Prog' (54 data points)
  Row 69: 'Actual Cumm. Prog' (54 data points)
  Row 70: 'Plan per Month' (54 data points)
  Row 71: 'Actual per Month' (54 data points)

=== Penta section ===
  Row 78: 'Achieved Mhrs' (51 data points)
  Row 79: 'Invoice Value (MUSD)' (35 data points)
  Row 80: 'Actual/VOWD' (40 data points)
  Row 81: 'POB' (53 data points)
  Row 82: 'Plan Cumm.Prog' (54 data points)
  Row 83: 'Actual Cumm. Prog' (54 data points)
  Row 84: 'Plan per Month' (54 data points)
  Row 85: 'Actual per Month' (54 data points)

=== Daewoo section ===
  Row 127: 'Achieved Mhrs' (51 data points)
  Row 128: 'Invoice V

In [16]:
# Extract FGRS data - find the row with most monthly numeric data near FGRS label
# We'll use adaptive row detection based on the scan above
def find_best_data_row(ws, keyword_rows, col_date_map):
    """Find the row with most numeric data near keyword rows."""
    best_row = None
    best_count = 0
    for kr in keyword_rows:
        for r in range(max(1, kr - 2), min(ws.max_row, kr + 5)):
            count = 0
            for col in col_date_map:
                v = ws.cell(row=r, column=col).value
                if isinstance(v, (int, float)):
                    count += 1
            if count > best_count:
                best_count = count
                best_row = r
    return best_row, best_count

# FGRS Monthly Cost
fgrs_row, fgrs_count = find_best_data_row(ws_main, fgrs_rows if fgrs_rows else [43], col_date_map)
print(f"FGRS data row: {fgrs_row} ({fgrs_count} data points)")
fgrs_data = extract_time_series(ws_main, fgrs_row, col_date_map) if fgrs_row else []
df_fgrs = pd.DataFrame(fgrs_data)
if len(df_fgrs) > 0:
    df_fgrs['cost_type'] = 'FGRS_RCE'
    df_fgrs.rename(columns={'value': 'monthly_amount_musd'}, inplace=True)
    print(f"FGRS monthly data: {len(df_fgrs)} points")
    print(df_fgrs.head())
else:
    print("WARNING: No FGRS data found")

FGRS data row: 49 (57 data points)
FGRS monthly data: 56 points
   year  month  monthly_amount_musd cost_type
0  2019      5            15.506369  FGRS_RCE
1  2019      6            33.998270  FGRS_RCE
2  2019      7            54.469964  FGRS_RCE
3  2019      8            76.816034  FGRS_RCE
4  2019      9           100.370881  FGRS_RCE


In [17]:
# LOGI RCE Cumulative Cost
logi_row, logi_count = find_best_data_row(ws_main, logi_rows if logi_rows else [49], col_date_map)
print(f"LOGI data row: {logi_row} ({logi_count} data points)")
logi_data = extract_time_series(ws_main, logi_row, col_date_map) if logi_row else []
df_logi = pd.DataFrame(logi_data)
if len(df_logi) > 0:
    df_logi['cost_type'] = 'LOGI_RCE'
    df_logi.rename(columns={'value': 'monthly_amount_musd'}, inplace=True)
    # LOGI may be cumulative - check if monotonically increasing
    is_cumulative = df_logi['monthly_amount_musd'].is_monotonic_increasing
    print(f"LOGI data: {len(df_logi)} points (cumulative={is_cumulative})")
    if is_cumulative:
        df_logi['cumulative_amount_musd'] = df_logi['monthly_amount_musd']
        df_logi['monthly_amount_musd'] = df_logi['cumulative_amount_musd'].diff().fillna(df_logi['cumulative_amount_musd'].iloc[0])
        print("Converted cumulative to monthly by differencing")
    print(df_logi.head())
else:
    print("WARNING: No LOGI data found")

LOGI data row: 49 (57 data points)
LOGI data: 56 points (cumulative=True)
Converted cumulative to monthly by differencing
   year  month  monthly_amount_musd cost_type  cumulative_amount_musd
0  2019      5            15.506369  LOGI_RCE               15.506369
1  2019      6            18.491901  LOGI_RCE               33.998270
2  2019      7            20.471694  LOGI_RCE               54.469964
3  2019      8            22.346070  LOGI_RCE               76.816034
4  2019      9            23.554847  LOGI_RCE              100.370881


In [18]:
# POB (Personnel on Board)
pob_row, pob_count = find_best_data_row(ws_main, pob_rows if pob_rows else [55], col_date_map)
print(f"POB data row: {pob_row} ({pob_count} data points)")
pob_data = extract_time_series(ws_main, pob_row, col_date_map) if pob_row else []
df_pob = pd.DataFrame(pob_data)
if len(df_pob) > 0:
    df_pob.rename(columns={'value': 'pob_count'}, inplace=True)
    print(f"POB data: {len(df_pob)} points, range: {df_pob['pob_count'].min():.0f} - {df_pob['pob_count'].max():.0f}")

# Isolation Facility
iso_row, iso_count = find_best_data_row(ws_main, isolation_rows if isolation_rows else [58], col_date_map)
print(f"\nIsolation data row: {iso_row} ({iso_count} data points)")
iso_data = extract_time_series(ws_main, iso_row, col_date_map) if iso_row else []
df_isolation = pd.DataFrame(iso_data)
if len(df_isolation) > 0:
    df_isolation.rename(columns={'value': 'isolation_count'}, inplace=True)
    print(f"Isolation data: {len(df_isolation)} points")

# Merge POB and Isolation
if len(df_pob) > 0:
    if len(df_isolation) > 0:
        df_pob_combined = df_pob.merge(df_isolation, on=['year', 'month'], how='left')
        df_pob_combined['isolation_count'] = df_pob_combined['isolation_count'].fillna(0)
    else:
        df_pob_combined = df_pob.copy()
        df_pob_combined['isolation_count'] = 0
    print(f"\nCombined POB data: {len(df_pob_combined)} rows")
    print(df_pob_combined.head())
else:
    df_pob_combined = pd.DataFrame(columns=['year', 'month', 'pob_count', 'isolation_count'])

POB data row: 68 (54 data points)
POB data: 54 points, range: 0 - 1

Isolation data row: 58 (10 data points)
Isolation data: 10 points

Combined POB data: 54 rows
   year  month  pob_count  isolation_count
0  2019      4   0.000000              0.0
1  2019      5   0.000000              0.0
2  2019      6   0.007505              0.0
3  2019      7   0.021632              0.0
4  2019      8   0.049963              0.0


### 1.6 Extract Subcontractor Monthly Data

In [19]:
# Define subcontractor metric labels to search for
SUBCON_METRICS = [
    'achieved_manhours', 'invoice_value_musd', 'actual_vowd_musd',
    'pob_plan', 'pob_actual',
    'cumulative_progress_plan', 'cumulative_progress_actual',
    'monthly_progress_plan', 'monthly_progress_actual'
]

# Common metric keywords in Excel labels
METRIC_KEYWORDS = {
    'mhr': 'achieved_manhours', 'manhour': 'achieved_manhours', 'man hour': 'achieved_manhours',
    'invoice': 'invoice_value_musd', 'inv ': 'invoice_value_musd',
    'vowd': 'actual_vowd_musd', 'vow': 'actual_vowd_musd',
    'pob': 'pob_actual', 'manpower': 'pob_actual',
    'plan pob': 'pob_plan', 'plan manpower': 'pob_plan',
    'cum': 'cumulative_progress_actual', 'cumulative': 'cumulative_progress_actual',
    'plan cum': 'cumulative_progress_plan', 'plan cumul': 'cumulative_progress_plan',
    'monthly prog': 'monthly_progress_actual', 'month prog': 'monthly_progress_actual',
    'plan month': 'monthly_progress_plan',
}

def classify_metric(label):
    """Classify a row label into a standard metric name."""
    label_lower = label.lower()
    # Check most specific keywords first
    for kw in sorted(METRIC_KEYWORDS.keys(), key=len, reverse=True):
        if kw in label_lower:
            return METRIC_KEYWORDS[kw]
    return None

def extract_subcontractor_data(ws, header_row, num_scan_rows=15):
    """Extract all metric rows for a subcontractor section."""
    results = []
    for r in range(header_row, min(ws.max_row + 1, header_row + num_scan_rows)):
        label_a = ws.cell(row=r, column=1).value
        label_b = ws.cell(row=r, column=2).value
        label_c = ws.cell(row=r, column=3).value
        label = str(label_a or label_b or label_c or '').strip()
        if not label:
            continue

        # Count data points
        data = extract_time_series(ws, r, col_date_map)
        if len(data) == 0:
            continue

        metric = classify_metric(label)
        if metric is None:
            metric = f'unknown_{r}'

        for pt in data:
            pt['metric'] = metric
            pt['excel_row'] = r
            pt['raw_label'] = label
        results.extend(data)
    return results

# Extract each subcontractor
all_subcon_data = []
subcon_sections = {
    'Meindo': meindo_rows[0] if meindo_rows else None,
    'Penta': penta_rows[0] if penta_rows else None,
    'Daewoo': daewoo_rows[0] if daewoo_rows else None,
}

for name, start_row in subcon_sections.items():
    if start_row is None:
        print(f"WARNING: {name} section not found, skipping")
        continue
    data = extract_subcontractor_data(ws_main, start_row)
    for d in data:
        d['subcontractor'] = name
    all_subcon_data.extend(data)
    metrics_found = set(d['metric'] for d in data)
    print(f"{name} (start row {start_row}): {len(data)} data points, metrics: {metrics_found}")

df_subcon_raw = pd.DataFrame(all_subcon_data)
if len(df_subcon_raw) > 0:
    print(f"\nTotal subcontractor records: {len(df_subcon_raw)}")
    print(df_subcon_raw.groupby(['subcontractor', 'metric']).size())
else:
    print("WARNING: No subcontractor data extracted")

Meindo (start row 63): 384 data points, metrics: {'achieved_manhours', 'cumulative_progress_plan', 'cumulative_progress_actual', 'pob_actual', 'unknown_71', 'invoice_value_musd', 'unknown_70', 'actual_vowd_musd'}
Penta (start row 77): 395 data points, metrics: {'achieved_manhours', 'cumulative_progress_plan', 'cumulative_progress_actual', 'pob_actual', 'invoice_value_musd', 'actual_vowd_musd', 'unknown_85', 'unknown_84'}
Daewoo (start row 126): 393 data points, metrics: {'achieved_manhours', 'unknown_135', 'cumulative_progress_plan', 'cumulative_progress_actual', 'unknown_134', 'pob_actual', 'invoice_value_musd', 'actual_vowd_musd', 'pob_plan'}

Total subcontractor records: 1172
subcontractor  metric                    
Daewoo         achieved_manhours             51
               actual_vowd_musd              36
               cumulative_progress_actual    54
               cumulative_progress_plan      54
               invoice_value_musd            35
               pob_actual     

### 1.7 Save Raw Extracted Data

In [20]:
raw_csvs = {
    'raw_contract_evolution.csv': df_evolution,
    'raw_pamf_claims.csv': df_pamf_raw,
}
if len(df_fgrs) > 0:
    raw_csvs['raw_fgrs_monthly.csv'] = df_fgrs
if len(df_logi) > 0:
    raw_csvs['raw_logi_monthly.csv'] = df_logi
if len(df_pob_combined) > 0:
    raw_csvs['raw_pob_monthly.csv'] = df_pob_combined
if len(df_subcon_raw) > 0:
    raw_csvs['raw_subcontractor_monthly.csv'] = df_subcon_raw

for filename, df in raw_csvs.items():
    path = os.path.join(OUTPUT_DIR, filename)
    df.to_csv(path, index=False)
    print(f"Saved: {filename} ({len(df)} rows)")

print(f"\nRaw CSVs saved to: {OUTPUT_DIR}")

Saved: raw_contract_evolution.csv (45 rows)
Saved: raw_pamf_claims.csv (95 rows)
Saved: raw_fgrs_monthly.csv (56 rows)
Saved: raw_logi_monthly.csv (56 rows)
Saved: raw_pob_monthly.csv (54 rows)
Saved: raw_subcontractor_monthly.csv (1172 rows)

Raw CSVs saved to: D:\BP\data_cleansing


---
## 2. DATA NORMALIZATION & TABLE CREATION
### Build all 13 target tables from extracted data

### 2.1 Master Tables

In [21]:
# === tb_m_project ===
df_tb_m_project = pd.DataFrame([{
    'project_id': 1,
    'project_code': 'TEP',
    'project_name': 'Tangguh Expansion Project',
    'client': 'BP Berau Ltd',
    'country': 'Indonesia',
    'contract_type': 'EPCI',
    'original_contract_value': 2432775726,
    'start_date': '2016-06-01',
    'planned_end_date': '2023-12-31',
    'actual_end_date': None,
    'status': 'In Progress'
}])
print("tb_m_project:")
print(df_tb_m_project.T)

tb_m_project:
                                                 0
project_id                                       1
project_code                                   TEP
project_name             Tangguh Expansion Project
client                                BP Berau Ltd
country                                  Indonesia
contract_type                                 EPCI
original_contract_value                 2432775726
start_date                              2016-06-01
planned_end_date                        2023-12-31
actual_end_date                               None
status                                 In Progress


In [22]:
# === tb_m_amendment ===
# Extract revised contract price row from evolution
revised_row = df_evolution[df_evolution['description'].str.contains('Revised CONTRACT PRICE|REVISED CONTRACT|CONTRACT PRICE', case=False, na=False)]
if len(revised_row) == 0:
    # Try last row which typically has totals
    revised_row = df_evolution.iloc[[-1]]
    print(f"Using last row as total: {revised_row['description'].values}")

print(f"Contract price row: {revised_row[['description', 'original_contract', 'amd_1', 'amd_2', 'amd_3', 'amd_4', 'amd_5']].values}")

# Also extract category totals (A, B, C, D)
cat_a = df_evolution[df_evolution['description'].str.contains('EPC LUMP SUM|LUMP SUM', case=False, na=False)]
cat_b = df_evolution[df_evolution['description'].str.contains('REIMBURSABLE', case=False, na=False)]
cat_c = df_evolution[df_evolution['description'].str.contains('PROVISIONAL SUM', case=False, na=False)]
cat_d = df_evolution[df_evolution['description'].str.contains('BACKCHARGE', case=False, na=False)]

def safe_get(df, col, default=0):
    if len(df) > 0 and col in df.columns:
        v = df.iloc[0][col]
        return v if v is not None and not (isinstance(v, float) and np.isnan(v)) else default
    return default

amendments = []
amd_cols = ['original_contract', 'amd_1', 'amd_2', 'amd_3', 'amd_4', 'amd_5']
amd_codes = ['ORIGINAL', 'AMD-1', 'AMD-2', 'AMD-3', 'AMD-4', 'AMD-5']
amd_names = ['Original Contract', 'Amendment 1', 'Amendment 2', 'Amendment 3', 'Amendment 4', 'Amendment 5']
amd_dates = ['2016-06-01', '2017-01-01', '2019-05-01', '2020-12-01', '2022-06-01', '2024-01-01']
amd_remarks = [
    'Original EPCI Contract Award',
    'Novated contract adjustment',
    'Scope restructure: VOs 1-9, logistics RCE',
    'COVID-19 response, FGRS RCE',
    'COVID FM costs, expanded reimbursable',
    'Final: COVID Tier 4, PP35, commissioning'
]

for i, col in enumerate(amd_cols):
    amendments.append({
        'amendment_id': i + 1,
        'project_id': 1,
        'amendment_code': amd_codes[i],
        'amendment_name': amd_names[i],
        'effective_date': amd_dates[i],
        'total_contract_value': safe_get(revised_row, col),
        'lump_sum_value': safe_get(cat_a, col),
        'reimbursable_value': safe_get(cat_b, col),
        'provisional_sum_value': safe_get(cat_c, col),
        'backcharge_value': safe_get(cat_d, col),
        'remarks': amd_remarks[i]
    })

df_tb_m_amendment = pd.DataFrame(amendments)
print("\ntb_m_amendment:")
print(df_tb_m_amendment[['amendment_code', 'total_contract_value', 'lump_sum_value', 'reimbursable_value', 'provisional_sum_value']].to_string())

Contract price row: [['Revised CONTRACT PRICE' 2432775726 2367252730 3140999999 3628379837
  4264450678 5047871388]]

tb_m_amendment:
  amendment_code  total_contract_value  lump_sum_value  reimbursable_value  provisional_sum_value
0       ORIGINAL            2432775726      2396417606                   0               36358120
1          AMD-1            2367252730      2330894610                   0               36358120
2          AMD-2            3140999999      2666580182           438061697               36358120
3          AMD-3            3628379837      2643421716           731743291              272958120
4          AMD-4            4264450678      2643421716          1196229990              444542262
5          AMD-5            5047871388      2643421716          1957442382              466750580


In [23]:
# === tb_m_cost_category ===
# Build hierarchical cost categories from evolution data
cost_cats = []
cat_id = 0
current_parent = None
current_type = None

for _, row in df_evolution.iterrows():
    rn = str(row['row_no']) if row['row_no'] else ''
    desc = str(row['description']).strip()

    # Skip the total row
    if 'Revised CONTRACT PRICE' in desc or 'REVISED' in desc.upper():
        continue

    # Level 1: Main categories (A, B, C, D)
    if rn in ('A', 'B', 'C', 'D'):
        cat_id += 1
        current_parent = cat_id
        if rn == 'A':
            current_type = 'LUMP_SUM'
        elif rn == 'B':
            current_type = 'REIMBURSABLE'
        elif rn == 'C':
            current_type = 'PROVISIONAL'
        elif rn == 'D':
            current_type = 'BACKCHARGE'
        cost_cats.append({
            'category_id': cat_id,
            'parent_category_id': None,
            'category_code': rn,
            'category_name': desc,
            'category_type': current_type,
            'level': 1
        })
    elif rn and current_parent:
        # Sub-categories
        cat_id += 1
        cost_cats.append({
            'category_id': cat_id,
            'parent_category_id': current_parent,
            'category_code': str(rn),
            'category_name': desc,
            'category_type': current_type,
            'level': 2
        })

df_tb_m_cost_category = pd.DataFrame(cost_cats)
print(f"tb_m_cost_category: {len(df_tb_m_cost_category)} categories")
print(df_tb_m_cost_category[['category_id', 'parent_category_id', 'category_code', 'category_name', 'category_type', 'level']].to_string())

tb_m_cost_category: 42 categories
    category_id  parent_category_id category_code                                category_name category_type  level
0             1                 NaN             A                                 EPC LUMP SUM      LUMP_SUM      1
1             2                 1.0             1                           PROJECT MANAGEMENT      LUMP_SUM      2
2             3                 1.0             2                                  ENGINEERING      LUMP_SUM      2
3             4                 1.0             3                                  PROCUREMENT      LUMP_SUM      2
4             5                 1.0           3.1                     Procurement of Equipment      LUMP_SUM      2
5             6                 1.0           3.2                 Procurement of Bulk Material      LUMP_SUM      2
6             7                 1.0           3.3         Capital Spare and Construction Spare      LUMP_SUM      2
7             8                 1.0   

In [24]:
# === tb_m_cost_discipline ===
# Extract unique disciplines from PAMF data
pamf_disciplines = df_pamf_raw[df_pamf_raw['level'] == 0]['label'].unique().tolist()
pamf_categories = df_pamf_raw[df_pamf_raw['level'] == 1]['label'].unique().tolist()

# Map discipline groups
def get_discipline_group(name):
    name_upper = name.upper()
    if 'COVID' in name_upper:
        return 'COVID'
    elif 'LOGISTIC' in name_upper:
        return 'LOGISTIC'
    elif 'PMT' in name_upper:
        return 'PMT'
    elif 'SMT' in name_upper:
        return 'SMT'
    return 'OTHER'

disciplines = []
disc_id = 0
for d in pamf_disciplines:
    if d == 'TOTAL':
        continue
    disc_id += 1
    disciplines.append({
        'discipline_id': disc_id,
        'discipline_code': f'{disc_id:02d}',
        'discipline_name': d,
        'discipline_group': get_discipline_group(d)
    })

# Add sub-disciplines from PAMF level 1
for cat in pamf_categories:
    if cat and cat not in [d['discipline_name'] for d in disciplines]:
        disc_id += 1
        disciplines.append({
            'discipline_id': disc_id,
            'discipline_code': f'{disc_id:02d}',
            'discipline_name': cat,
            'discipline_group': 'SUB'
        })

df_tb_m_cost_discipline = pd.DataFrame(disciplines)
print(f"tb_m_cost_discipline: {len(df_tb_m_cost_discipline)} disciplines")
print(df_tb_m_cost_discipline.to_string())

tb_m_cost_discipline: 15 disciplines
    discipline_id discipline_code            discipline_name discipline_group
0               1              01                      covid            COVID
1               2              02                   logistic         LOGISTIC
2               3              03                        PMT              PMT
3               4              04                        SMT              SMT
4               5              05         09 COVID-19 IMPACT              SUB
5               6              06      BP ADDITIONAL REQUEST              SUB
6               7              07                   LOGISTIC              SUB
7               8              08      01 PROJECT MANAGEMENT              SUB
8               9              09             02 ENGINEERING              SUB
9              10              10  04 CONSTRUCTION PERMANENT              SUB
10             11              11         05 TEMPORARY WORKS              SUB
11             12          

In [25]:
# === tb_m_subcontractor ===
df_tb_m_subcontractor = pd.DataFrame([
    {'subcontractor_id': 1, 'project_id': 1, 'subcontractor_name': 'Meindo',
     'contract_number': '', 'scope_of_work': 'Piping Erection',
     'contract_value': None, 'start_date': None, 'end_date': None},
    {'subcontractor_id': 2, 'project_id': 1, 'subcontractor_name': 'Penta',
     'contract_number': '', 'scope_of_work': 'Piping Erection',
     'contract_value': None, 'start_date': None, 'end_date': None},
    {'subcontractor_id': 3, 'project_id': 1, 'subcontractor_name': 'Daewoo',
     'contract_number': '1306754', 'scope_of_work': 'General Construction / Piping Erection',
     'contract_value': None, 'start_date': None, 'end_date': None},
])
print("tb_m_subcontractor:")
print(df_tb_m_subcontractor.to_string())

tb_m_subcontractor:
   subcontractor_id  project_id subcontractor_name contract_number                           scope_of_work contract_value start_date end_date
0                 1           1             Meindo                                         Piping Erection           None       None     None
1                 2           1              Penta                                         Piping Erection           None       None     None
2                 3           1             Daewoo         1306754  General Construction / Piping Erection           None       None     None


In [26]:
# === tb_m_event ===
df_tb_m_event = pd.DataFrame([
    {'event_id': 1, 'project_id': 1, 'event_code': 'CONTRACT_AWARD',
     'event_name': 'Contract Award', 'event_type': 'MILESTONE',
     'start_date': '2016-06-01', 'end_date': None,
     'description': 'Original EPCI Contract signed'},
    {'event_id': 2, 'project_id': 1, 'event_code': 'AMD_1',
     'event_name': 'Amendment 1', 'event_type': 'AMENDMENT',
     'start_date': '2017-01-01', 'end_date': None,
     'description': 'Novated contract adjustment (GE contract)'},
    {'event_id': 3, 'project_id': 1, 'event_code': 'AMD_2',
     'event_name': 'Amendment 2', 'event_type': 'AMENDMENT',
     'start_date': '2019-05-01', 'end_date': None,
     'description': 'Major scope restructure: reimbursable, VOs, logistics'},
    {'event_id': 4, 'project_id': 1, 'event_code': 'COVID_PANDEMIC',
     'event_name': 'COVID-19 Pandemic Onset', 'event_type': 'PANDEMIC',
     'start_date': '2020-03-01', 'end_date': None,
     'description': 'COVID-19 begins affecting project, POB restricted 12000+ to ~6300'},
    {'event_id': 5, 'project_id': 1, 'event_code': 'FM',
     'event_name': 'Force Majeure Declaration', 'event_type': 'FORCE_MAJEURE',
     'start_date': '2020-12-01', 'end_date': None,
     'description': 'Force Majeure declared due to COVID-19 impact'},
    {'event_id': 6, 'project_id': 1, 'event_code': 'AMD_3',
     'event_name': 'Amendment 3', 'event_type': 'AMENDMENT',
     'start_date': '2020-12-01', 'end_date': None,
     'description': 'COVID response, FGRS RCE, additional provisional sums'},
    {'event_id': 7, 'project_id': 1, 'event_code': 'OUTBREAK_2',
     'event_name': '2nd COVID Outbreak (Delta)', 'event_type': 'OUTBREAK',
     'start_date': '2021-07-01', 'end_date': '2021-12-31',
     'description': 'Second COVID outbreak at Tangguh site (Delta variant)'},
    {'event_id': 8, 'project_id': 1, 'event_code': 'AMD_4',
     'event_name': 'Amendment 4', 'event_type': 'AMENDMENT',
     'start_date': '2022-06-01', 'end_date': None,
     'description': 'Extended COVID FM costs, expanded reimbursable'},
    {'event_id': 9, 'project_id': 1, 'event_code': 'OUTBREAK_3',
     'event_name': '3rd COVID Outbreak (Omicron)', 'event_type': 'OUTBREAK',
     'start_date': '2022-01-01', 'end_date': '2022-06-30',
     'description': 'Third COVID outbreak (Omicron variant)'},
    {'event_id': 10, 'project_id': 1, 'event_code': 'AMD_5',
     'event_name': 'Amendment 5', 'event_type': 'AMENDMENT',
     'start_date': '2024-01-01', 'end_date': None,
     'description': 'Final amendment: COVID Tier 4, labor law PP35, commissioning'},
])
print("tb_m_event:")
print(df_tb_m_event[['event_id', 'event_code', 'event_name', 'event_type', 'start_date']].to_string())

tb_m_event:
   event_id      event_code                    event_name     event_type  start_date
0         1  CONTRACT_AWARD                Contract Award      MILESTONE  2016-06-01
1         2           AMD_1                   Amendment 1      AMENDMENT  2017-01-01
2         3           AMD_2                   Amendment 2      AMENDMENT  2019-05-01
3         4  COVID_PANDEMIC       COVID-19 Pandemic Onset       PANDEMIC  2020-03-01
4         5              FM     Force Majeure Declaration  FORCE_MAJEURE  2020-12-01
5         6           AMD_3                   Amendment 3      AMENDMENT  2020-12-01
6         7      OUTBREAK_2    2nd COVID Outbreak (Delta)       OUTBREAK  2021-07-01
7         8           AMD_4                   Amendment 4      AMENDMENT  2022-06-01
8         9      OUTBREAK_3  3rd COVID Outbreak (Omicron)       OUTBREAK  2022-01-01
9        10           AMD_5                   Amendment 5      AMENDMENT  2024-01-01


### 2.2 Transaction Tables

In [27]:
# === tb_t_contract_value ===
# Melt evolution from wide to long format: one row per (amendment, line_item)
amd_col_map = {
    'original_contract': 1, 'amd_1': 2, 'amd_2': 3,
    'amd_3': 4, 'amd_4': 5, 'amd_5': 6
}

# Assign cost_category_id based on section hierarchy
df_evo_work = df_evolution.copy()
current_cat = None
categories = []
for _, row in df_evo_work.iterrows():
    rn = str(row['row_no']) if row['row_no'] else ''
    if rn == 'A':
        current_cat = 1
    elif rn == 'B':
        current_cat = 2
    elif rn == 'C':
        current_cat = 3
    elif rn == 'D':
        current_cat = 4
    categories.append(current_cat)
df_evo_work['cost_category_id'] = categories

# Skip the total row
df_evo_work = df_evo_work[~df_evo_work['description'].str.contains('Revised CONTRACT PRICE|REVISED CONTRACT', case=False, na=False)]

# Melt
melted = df_evo_work.melt(
    id_vars=['row_no', 'description', 'cost_category_id'],
    value_vars=list(amd_col_map.keys()),
    var_name='amendment_col', value_name='amount_usd'
)
melted['amendment_id'] = melted['amendment_col'].map(amd_col_map)
melted = melted.dropna(subset=['cost_category_id'])
# Keep rows even with null amounts (shows what was added in later amendments)

df_tb_t_contract_value = melted[['amendment_id', 'cost_category_id', 'description', 'amount_usd']].copy()
df_tb_t_contract_value.insert(0, 'id', range(1, len(df_tb_t_contract_value) + 1))
df_tb_t_contract_value['remarks'] = None

print(f"tb_t_contract_value: {len(df_tb_t_contract_value)} rows")
print(f"By amendment: {df_tb_t_contract_value.groupby('amendment_id').size().to_dict()}")
print(df_tb_t_contract_value.head(20).to_string())

tb_t_contract_value: 258 rows
By amendment: {1: 43, 2: 43, 3: 43, 4: 43, 5: 43, 6: 43}
    id  amendment_id  cost_category_id                           description  amount_usd remarks
1    1             1               1.0                          EPC LUMP SUM  2396417606    None
2    2             1               1.0                    PROJECT MANAGEMENT   157861778    None
3    3             1               1.0                           ENGINEERING   149829449    None
4    4             1               1.0                           PROCUREMENT   606018791    None
5    5             1               1.0              Procurement of Equipment        None    None
6    6             1               1.0          Procurement of Bulk Material        None    None
7    7             1               1.0  Capital Spare and Construction Spare        None    None
8    8             1               1.0                          CONSTRUCTION   877712530    None
9    9             1               1.0  

In [28]:
# === tb_t_monthly_cost ===
cost_frames = []
if len(df_fgrs) > 0:
    cost_frames.append(df_fgrs[['year', 'month', 'cost_type', 'monthly_amount_musd']].copy())
if len(df_logi) > 0:
    logi_for_cost = df_logi[['year', 'month', 'cost_type', 'monthly_amount_musd']].copy()
    if 'cumulative_amount_musd' in df_logi.columns:
        logi_for_cost['cumulative_amount_musd'] = df_logi['cumulative_amount_musd'].values
    cost_frames.append(logi_for_cost)

if cost_frames:
    df_tb_t_monthly_cost = pd.concat(cost_frames, ignore_index=True)
    df_tb_t_monthly_cost.insert(0, 'id', range(1, len(df_tb_t_monthly_cost) + 1))
    df_tb_t_monthly_cost['project_id'] = 1
    # Add cumulative for FGRS
    fgrs_mask = df_tb_t_monthly_cost['cost_type'] == 'FGRS_RCE'
    if fgrs_mask.any():
        fgrs_sorted = df_tb_t_monthly_cost[fgrs_mask].sort_values(['year', 'month'])
        df_tb_t_monthly_cost.loc[fgrs_mask, 'cumulative_amount_musd'] = fgrs_sorted['monthly_amount_musd'].cumsum().values
    if 'cumulative_amount_musd' not in df_tb_t_monthly_cost.columns:
        df_tb_t_monthly_cost['cumulative_amount_musd'] = None
    print(f"tb_t_monthly_cost: {len(df_tb_t_monthly_cost)} rows")
    print(df_tb_t_monthly_cost.groupby('cost_type').size())
    print(df_tb_t_monthly_cost.head())
else:
    df_tb_t_monthly_cost = pd.DataFrame()
    print("WARNING: No monthly cost data")

tb_t_monthly_cost: 112 rows
cost_type
FGRS_RCE    56
LOGI_RCE    56
dtype: int64
   id  year  month cost_type  monthly_amount_musd  cumulative_amount_musd  \
0   1  2019      5  FGRS_RCE            15.506369               15.506369   
1   2  2019      6  FGRS_RCE            33.998270               49.504638   
2   3  2019      7  FGRS_RCE            54.469964              103.974602   
3   4  2019      8  FGRS_RCE            76.816034              180.790636   
4   5  2019      9  FGRS_RCE           100.370881              281.161517   

   project_id  
0           1  
1           1  
2           1  
3           1  
4           1  


In [29]:
# === tb_t_monthly_pob ===
if len(df_pob_combined) > 0:
    df_tb_t_monthly_pob = df_pob_combined.copy()
    df_tb_t_monthly_pob.insert(0, 'id', range(1, len(df_tb_t_monthly_pob) + 1))
    df_tb_t_monthly_pob['project_id'] = 1
    df_tb_t_monthly_pob['remarks'] = None
    print(f"tb_t_monthly_pob: {len(df_tb_t_monthly_pob)} rows")
    print(f"POB range: {df_tb_t_monthly_pob['pob_count'].min():.0f} - {df_tb_t_monthly_pob['pob_count'].max():.0f}")
    print(df_tb_t_monthly_pob.head())
else:
    df_tb_t_monthly_pob = pd.DataFrame()
    print("WARNING: No POB data")

tb_t_monthly_pob: 54 rows
POB range: 0 - 1
   id  year  month  pob_count  isolation_count  project_id remarks
0   1  2019      4   0.000000              0.0           1    None
1   2  2019      5   0.000000              0.0           1    None
2   3  2019      6   0.007505              0.0           1    None
3   4  2019      7   0.021632              0.0           1    None
4   5  2019      8   0.049963              0.0           1    None


In [30]:
# === tb_t_pamf_claim ===
# Use leaf-level (level 2) entries, plus level 0/1 summaries
pamf_for_table = df_pamf_raw[df_pamf_raw['level'].isin([0, 1, 2])].copy()
pamf_for_table = pamf_for_table[pamf_for_table['label'] != 'TOTAL']

# Map discipline to discipline_id
disc_map = dict(zip(df_tb_m_cost_discipline['discipline_name'], df_tb_m_cost_discipline['discipline_id']))

pamf_for_table['discipline_id'] = pamf_for_table['discipline'].map(disc_map)
pamf_for_table.insert(0, 'id', range(1, len(pamf_for_table) + 1))
pamf_for_table['project_id'] = 1

# Determine pamf_group
def get_pamf_group(row):
    d = str(row.get('discipline', '')).upper()
    if 'COVID' in d:
        return 'COVID'
    elif 'LOGISTIC' in d:
        return 'LOGISTIC'
    elif 'PMT' in d:
        return 'PMT'
    elif 'SMT' in d:
        return 'SMT'
    return 'OTHER'

pamf_for_table['pamf_group'] = pamf_for_table.apply(get_pamf_group, axis=1)

df_tb_t_pamf_claim = pamf_for_table[['id', 'project_id', 'discipline_id', 'discipline',
    'pamf_group', 'label', 'level', 'claim_count', 'claim_amount_usd']].copy()
df_tb_t_pamf_claim.rename(columns={'claim_count': 'pamf_count'}, inplace=True)

print(f"tb_t_pamf_claim: {len(df_tb_t_pamf_claim)} rows")
print(f"Total claim amount: ${df_tb_t_pamf_claim[df_tb_t_pamf_claim['level'] == 0]['claim_amount_usd'].sum():,.2f}")
print(df_tb_t_pamf_claim.head(10).to_string())

tb_t_pamf_claim: 94 rows
Total claim amount: $1,771,897,841.51
   id  project_id  discipline_id discipline pamf_group                       label  level  pamf_count  claim_amount_usd
0   1           1              1      covid      COVID                       covid      0          45      9.011710e+07
1   2           1              1      covid      COVID          09 COVID-19 IMPACT      1          45      9.011710e+07
2   3           1              1      covid      COVID           VO COVID OUTBREAK      2           2      4.124617e+05
3   4           1              1      covid      COVID          VO11 : DIRECT COST      2          43      8.970464e+07
4   5           1              2   logistic   LOGISTIC                    logistic      0         342      1.569896e+08
5   6           1              2   logistic   LOGISTIC       BP ADDITIONAL REQUEST      1           2      7.932431e+05
6   7           1              2   logistic   LOGISTIC    01 BP ADDITIONAL REQUEST      2        

In [31]:
# === tb_t_variation_order ===
vo_rows = df_evolution[df_evolution['description'].str.contains('VO|Variation|variation', case=False, na=False)]
print(f"VO-related rows found: {len(vo_rows)}")
print(vo_rows[['row_no', 'description']].to_string())

vo_records = []
vo_id = 0
for _, row in vo_rows.iterrows():
    for amd_col, amd_id in amd_col_map.items():
        val = row[amd_col]
        if val is not None and isinstance(val, (int, float)) and not np.isnan(val):
            vo_id += 1
            vo_records.append({
                'vo_id': vo_id,
                'project_id': 1,
                'vo_number': str(row['row_no']) if row['row_no'] else '',
                'vo_name': row['description'],
                'amount_usd': val,
                'status': 'APPROVED',
                'approved_in_amendment': amd_id,
                'approved_date': None
            })

df_tb_t_variation_order = pd.DataFrame(vo_records)
print(f"\ntb_t_variation_order: {len(df_tb_t_variation_order)} rows")
if len(df_tb_t_variation_order) > 0:
    print(df_tb_t_variation_order.to_string())

VO-related rows found: 4
   row_no                                  description
19     10                  APPROVED VOs (VO No. 1 - 9)
20     11                     APPROVED VOs (VO No. 10)
21     12  Existing VORs agreed as part of AMENDMENT 3
31      6                           List of VARIATIONS

tb_t_variation_order: 12 rows
    vo_id  project_id vo_number                                      vo_name  amount_usd    status  approved_in_amendment approved_date
0       1           1        10                  APPROVED VOs (VO No. 1 - 9)    26358170  APPROVED                      3          None
1       2           1        10                  APPROVED VOs (VO No. 1 - 9)    26358170  APPROVED                      4          None
2       3           1        10                  APPROVED VOs (VO No. 1 - 9)    26358170  APPROVED                      5          None
3       4           1        10                  APPROVED VOs (VO No. 1 - 9)    26358170  APPROVED                      6    

In [32]:
# === tb_t_subcontractor_monthly ===
subcon_id_map = {'Meindo': 1, 'Penta': 2, 'Daewoo': 3}

if len(df_subcon_raw) > 0:
    df_tb_t_subcontractor_monthly = df_subcon_raw.copy()
    df_tb_t_subcontractor_monthly['subcontractor_id'] = df_tb_t_subcontractor_monthly['subcontractor'].map(subcon_id_map)
    df_tb_t_subcontractor_monthly.insert(0, 'id', range(1, len(df_tb_t_subcontractor_monthly) + 1))

    # Pivot to wide format per (subcontractor, year, month) with metric columns
    pivot_data = df_tb_t_subcontractor_monthly.pivot_table(
        index=['subcontractor_id', 'subcontractor', 'year', 'month'],
        columns='metric', values='value', aggfunc='first'
    ).reset_index()
    pivot_data.insert(0, 'id', range(1, len(pivot_data) + 1))

    print(f"tb_t_subcontractor_monthly (long): {len(df_tb_t_subcontractor_monthly)} rows")
    print(f"tb_t_subcontractor_monthly (wide/pivoted): {len(pivot_data)} rows")
    print(f"Columns: {list(pivot_data.columns)}")
    # Use long format for storage
else:
    df_tb_t_subcontractor_monthly = pd.DataFrame()
    pivot_data = pd.DataFrame()
    print("WARNING: No subcontractor monthly data")

tb_t_subcontractor_monthly (long): 1172 rows
tb_t_subcontractor_monthly (wide/pivoted): 165 rows
Columns: ['id', 'subcontractor_id', 'subcontractor', 'year', 'month', 'achieved_manhours', 'actual_vowd_musd', 'cumulative_progress_actual', 'cumulative_progress_plan', 'invoice_value_musd', 'pob_actual', 'pob_plan', 'unknown_134', 'unknown_135', 'unknown_70', 'unknown_71', 'unknown_84', 'unknown_85']


In [33]:
# === tb_t_project_progress ===
if len(df_subcon_raw) > 0:
    # Extract progress metrics from subcontractor data
    progress_metrics = [m for m in df_subcon_raw['metric'].unique()
                       if 'progress' in m.lower() or 'cum' in m.lower()]
    print(f"Progress metrics found: {progress_metrics}")

    df_progress = df_subcon_raw[df_subcon_raw['metric'].isin(progress_metrics)].copy()

    if len(df_progress) > 0:
        # Pivot to get plan vs actual columns
        df_progress_pivot = df_progress.pivot_table(
            index=['subcontractor', 'year', 'month'],
            columns='metric', values='value', aggfunc='first'
        ).reset_index()

        df_progress_pivot['subcontractor_id'] = df_progress_pivot['subcontractor'].map(subcon_id_map)
        df_progress_pivot.insert(0, 'id', range(1, len(df_progress_pivot) + 1))
        df_progress_pivot['project_id'] = 1

        # Rename columns for clarity
        col_rename = {}
        for c in df_progress_pivot.columns:
            if 'plan' in str(c).lower() and 'cum' in str(c).lower():
                col_rename[c] = 'plan_progress_pct'
            elif 'actual' in str(c).lower() and 'cum' in str(c).lower():
                col_rename[c] = 'overall_progress_pct'
        df_progress_pivot.rename(columns=col_rename, inplace=True)

        df_tb_t_project_progress = df_progress_pivot
        print(f"tb_t_project_progress: {len(df_tb_t_project_progress)} rows")
        print(df_tb_t_project_progress.head())
    else:
        df_tb_t_project_progress = pd.DataFrame()
        print("No progress metrics found in subcontractor data")
else:
    df_tb_t_project_progress = pd.DataFrame()
    print("WARNING: No project progress data")

Progress metrics found: ['cumulative_progress_plan', 'cumulative_progress_actual']
tb_t_project_progress: 162 rows
metric  id subcontractor  year  month  overall_progress_pct  \
0        1        Daewoo  2019      4                   0.0   
1        2        Daewoo  2019      5                   0.0   
2        3        Daewoo  2019      6                   0.0   
3        4        Daewoo  2019      7                   0.0   
4        5        Daewoo  2019      8                   0.0   

metric  plan_progress_pct  subcontractor_id  project_id  
0                0.000000                 3           1  
1                0.001632                 3           1  
2                0.043625                 3           1  
3                0.049906                 3           1  
4                0.070925                 3           1  


---
## 3. DATA CLEANSING & VALIDATION

In [34]:
# Collect all tables
all_tables = {
    'tb_m_project': df_tb_m_project,
    'tb_m_amendment': df_tb_m_amendment,
    'tb_m_cost_category': df_tb_m_cost_category,
    'tb_m_cost_discipline': df_tb_m_cost_discipline,
    'tb_m_subcontractor': df_tb_m_subcontractor,
    'tb_m_event': df_tb_m_event,
    'tb_t_contract_value': df_tb_t_contract_value,
    'tb_t_monthly_cost': df_tb_t_monthly_cost,
    'tb_t_monthly_pob': df_tb_t_monthly_pob,
    'tb_t_pamf_claim': df_tb_t_pamf_claim,
    'tb_t_variation_order': df_tb_t_variation_order,
    'tb_t_subcontractor_monthly': df_tb_t_subcontractor_monthly,
    'tb_t_project_progress': df_tb_t_project_progress,
}

# Remove empty tables
all_tables = {k: v for k, v in all_tables.items() if len(v) > 0}

print(f"{'Table':<35} {'Rows':>6} {'Cols':>5} {'Nulls':>8}")
print("=" * 60)
for name, df in all_tables.items():
    null_count = df.isnull().sum().sum()
    print(f"{name:<35} {len(df):>6} {len(df.columns):>5} {null_count:>8}")

Table                                 Rows  Cols    Nulls
tb_m_project                             1    11        1
tb_m_amendment                           6    11        0
tb_m_cost_category                      42     6        4
tb_m_cost_discipline                    15     4        0
tb_m_subcontractor                       3     8        9
tb_m_event                              10     8        8
tb_t_contract_value                    258     6      349
tb_t_monthly_cost                      112     7        0
tb_t_monthly_pob                        54     7       54
tb_t_pamf_claim                         94     9        1
tb_t_variation_order                    12     8       12
tb_t_subcontractor_monthly            1172     9        0
tb_t_project_progress                  162     8        0


In [35]:
# === Data Cleansing Operations ===
print("=== Cleansing Operations ===\n")

# 1. Strip whitespace from text columns
for tname, df in all_tables.items():
    for col in df.select_dtypes(include='object').columns:
        if hasattr(df[col], 'str'):
            all_tables[tname][col] = df[col].str.strip()
    print(f"[{tname}] Text columns stripped")

# 2. Standardize numeric columns
for tname in ['tb_t_monthly_cost']:
    if tname in all_tables:
        df = all_tables[tname]
        for col in df.select_dtypes(include='float64').columns:
            if 'amount' in col or 'musd' in col:
                df[col] = df[col].round(6)
        print(f"[{tname}] Financial columns rounded")

# 3. POB as integers
if 'tb_t_monthly_pob' in all_tables:
    df = all_tables['tb_t_monthly_pob']
    for col in ['pob_count', 'isolation_count']:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
    print("[tb_t_monthly_pob] POB counts cast to int")

# 4. Validate contract values are positive where expected
if 'tb_t_contract_value' in all_tables:
    df = all_tables['tb_t_contract_value']
    neg_vals = df[df['amount_usd'].apply(lambda x: isinstance(x, (int, float)) and x < 0)]
    if len(neg_vals) > 0:
        print(f"[tb_t_contract_value] Negative values found (expected for backcharges): {len(neg_vals)}")

# 5. Check PAMF claim amounts
if 'tb_t_pamf_claim' in all_tables:
    df = all_tables['tb_t_pamf_claim']
    total = df[df['level'] == 0]['claim_amount_usd'].sum()
    print(f"[tb_t_pamf_claim] Total claim (level 0): ${total:,.2f}")

print("\nCleansing complete.")

=== Cleansing Operations ===

[tb_m_project] Text columns stripped
[tb_m_amendment] Text columns stripped
[tb_m_cost_category] Text columns stripped
[tb_m_cost_discipline] Text columns stripped
[tb_m_subcontractor] Text columns stripped
[tb_m_event] Text columns stripped
[tb_t_contract_value] Text columns stripped
[tb_t_monthly_cost] Text columns stripped
[tb_t_monthly_pob] Text columns stripped
[tb_t_pamf_claim] Text columns stripped
[tb_t_variation_order] Text columns stripped
[tb_t_subcontractor_monthly] Text columns stripped
[tb_t_project_progress] Text columns stripped
[tb_t_monthly_cost] Financial columns rounded
[tb_t_monthly_pob] POB counts cast to int
[tb_t_contract_value] Negative values found (expected for backcharges): 6
[tb_t_pamf_claim] Total claim (level 0): $1,771,897,841.51

Cleansing complete.


In [36]:
# === Referential Integrity Checks ===
print("=== Referential Integrity Checks ===\n")

# tb_t_contract_value.amendment_id -> tb_m_amendment.amendment_id
if 'tb_t_contract_value' in all_tables and 'tb_m_amendment' in all_tables:
    cv_amds = set(all_tables['tb_t_contract_value']['amendment_id'].dropna().unique())
    m_amds = set(all_tables['tb_m_amendment']['amendment_id'].unique())
    orphaned = cv_amds - m_amds
    print(f"contract_value.amendment_id: {cv_amds}")
    print(f"  Valid: {cv_amds & m_amds}, Orphaned: {orphaned}")

# tb_t_subcontractor_monthly FK
if 'tb_t_subcontractor_monthly' in all_tables and 'tb_m_subcontractor' in all_tables:
    sc_ids = set(all_tables['tb_t_subcontractor_monthly']['subcontractor_id'].dropna().unique())
    m_sc = set(all_tables['tb_m_subcontractor']['subcontractor_id'].unique())
    print(f"\nsubcontractor_monthly.subcontractor_id: {sc_ids}")
    print(f"  Valid: {sc_ids & m_sc}, Orphaned: {sc_ids - m_sc}")

# Cross-check amendment totals
if 'tb_m_amendment' in all_tables:
    print("\nAmendment value summary:")
    for _, amd in all_tables['tb_m_amendment'].iterrows():
        print(f"  {amd['amendment_code']}: Total={amd['total_contract_value']:>15,.0f} | "
              f"LS={amd['lump_sum_value']:>15,.0f} | "
              f"Reimb={amd['reimbursable_value']:>15,.0f} | "
              f"Prov={amd['provisional_sum_value']:>15,.0f}")

print("\nAll checks complete.")

=== Referential Integrity Checks ===

contract_value.amendment_id: {1, 2, 3, 4, 5, 6}
  Valid: {1, 2, 3, 4, 5, 6}, Orphaned: set()

subcontractor_monthly.subcontractor_id: {1, 2, 3}
  Valid: {1, 2, 3}, Orphaned: set()

Amendment value summary:
  ORIGINAL: Total=  2,432,775,726 | LS=  2,396,417,606 | Reimb=              0 | Prov=     36,358,120
  AMD-1: Total=  2,367,252,730 | LS=  2,330,894,610 | Reimb=              0 | Prov=     36,358,120
  AMD-2: Total=  3,140,999,999 | LS=  2,666,580,182 | Reimb=    438,061,697 | Prov=     36,358,120
  AMD-3: Total=  3,628,379,837 | LS=  2,643,421,716 | Reimb=    731,743,291 | Prov=    272,958,120
  AMD-4: Total=  4,264,450,678 | LS=  2,643,421,716 | Reimb=  1,196,229,990 | Prov=    444,542,262
  AMD-5: Total=  5,047,871,388 | LS=  2,643,421,716 | Reimb=  1,957,442,382 | Prov=    466,750,580

All checks complete.


---
## 4. EXPLORATORY DATA ANALYSIS & VISUALIZATIONS
### 4.1 Contract Value Growth Analysis

In [37]:
# Contract Value Evolution Bar Chart
if 'tb_m_amendment' in all_tables:
    fig, axes = plt.subplots(1, 2, figsize=(18, 7))

    df_amd = all_tables['tb_m_amendment']
    labels = df_amd['amendment_code'].values
    values = df_amd['total_contract_value'].values / 1e9

    colors = ['#2196F3', '#64B5F6', '#4CAF50', '#FF9800', '#F44336', '#9C27B0'][:len(labels)]
    bars = axes[0].bar(labels, values, color=colors, edgecolor='black', linewidth=0.5)
    axes[0].set_title('TEP Contract Value Evolution', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Contract Value (Billion USD)')
    axes[0].set_ylim(0, max(values) * 1.15)
    for bar, val in zip(bars, values):
        axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
                     f'${val:.2f}B', ha='center', fontsize=9, fontweight='bold')

    # Stacked bar: cost category breakdown per amendment
    cat_data = {}
    for cat_name, cat_col in [('Lump Sum', 'lump_sum_value'), ('Reimbursable', 'reimbursable_value'),
                               ('Provisional', 'provisional_sum_value')]:
        cat_data[cat_name] = df_amd[cat_col].values / 1e9

    x = np.arange(len(labels))
    width = 0.6
    bottom = np.zeros(len(labels))
    cat_colors = ['#1565C0', '#43A047', '#EF6C00']

    for (cat_name, vals), color in zip(cat_data.items(), cat_colors):
        vals_safe = np.nan_to_num(vals, nan=0.0)
        axes[1].bar(x, vals_safe, width, bottom=bottom, label=cat_name, color=color, alpha=0.85)
        bottom += vals_safe

    axes[1].set_xticks(x)
    axes[1].set_xticklabels(labels)
    axes[1].set_title('Contract Composition by Cost Category', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Amount (Billion USD)')
    axes[1].legend(title='Category')

    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'viz_01_contract_evolution.png'), dpi=150, bbox_inches='tight')
    plt.show()
    print("Saved: viz_01_contract_evolution.png")
else:
    print("Skipped: no amendment data")

Saved: viz_01_contract_evolution.png


### 4.2 Cost Category Breakdown

In [38]:
if 'tb_m_amendment' in all_tables and 'tb_t_pamf_claim' in all_tables:
    fig, axes = plt.subplots(1, 2, figsize=(16, 7))

    # AMD-5 (last amendment) breakdown donut
    df_amd = all_tables['tb_m_amendment']
    last_amd = df_amd.iloc[-1]
    cat_labels = ['Lump Sum', 'Reimbursable', 'Provisional']
    cat_vals = [abs(last_amd['lump_sum_value'] or 0),
                abs(last_amd['reimbursable_value'] or 0),
                abs(last_amd['provisional_sum_value'] or 0)]
    cat_vals_b = [v / 1e9 for v in cat_vals]

    if last_amd['backcharge_value'] and abs(last_amd['backcharge_value']) > 0:
        cat_labels.append('Backcharge')
        cat_vals_b.append(abs(last_amd['backcharge_value']) / 1e9)

    colors_pie = ['#1976D2', '#388E3C', '#F57C00', '#D32F2F'][:len(cat_labels)]
    wedges, texts, autotexts = axes[0].pie(cat_vals_b, labels=cat_labels, autopct='%1.1f%%',
        colors=colors_pie, startangle=90, pctdistance=0.85, wedgeprops=dict(width=0.4))
    total_b = sum(cat_vals_b)
    axes[0].set_title(f'{last_amd["amendment_code"]} Cost Split (${total_b:.2f}B)', fontsize=13, fontweight='bold')

    # PAMF claims by discipline
    df_pamf = all_tables['tb_t_pamf_claim']
    disc_summary = df_pamf[df_pamf['level'] == 0].groupby('pamf_group').agg(
        total=('claim_amount_usd', 'sum')
    ).sort_values('total', ascending=True)

    disc_colors = {'COVID': '#E53935', 'LOGISTIC': '#1E88E5', 'PMT': '#43A047', 'SMT': '#FB8C00', 'OTHER': '#757575'}
    bar_colors = [disc_colors.get(g, '#757575') for g in disc_summary.index]

    axes[1].barh(disc_summary.index, disc_summary['total'] / 1e6, color=bar_colors)
    axes[1].set_xlabel('Claim Amount (Million USD)')
    axes[1].set_title('PAMF Claims by Discipline Group', fontsize=13, fontweight='bold')
    for i, (idx, row) in enumerate(disc_summary.iterrows()):
        axes[1].text(row['total'] / 1e6 + 5, i, f'${row["total"]/1e6:.0f}M', va='center', fontsize=10)

    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'viz_02_cost_breakdown.png'), dpi=150, bbox_inches='tight')
    plt.show()
    print("Saved: viz_02_cost_breakdown.png")

Saved: viz_02_cost_breakdown.png


### 4.3 Monthly Cost Time-Series

In [40]:
if 'tb_t_monthly_cost' in all_tables and len(all_tables['tb_t_monthly_cost']) > 0:
    fig, axes = plt.subplots(2, 1, figsize=(18, 10))

    df_mc = all_tables['tb_t_monthly_cost']

    # ======================
    # FGRS
    # ======================
    fgrs = df_mc[df_mc['cost_type'] == 'FGRS_RCE'].copy()
    if not fgrs.empty:
        fgrs['date'] = pd.to_datetime(
            fgrs['year'].astype(int).astype(str)
            + '-'
            + fgrs['month'].astype(int).astype(str).str.zfill(2)
            + '-01'
        )
        fgrs = fgrs.sort_values('date')

        x = fgrs['date'].to_numpy()
        y = fgrs['monthly_amount_musd'].to_numpy()

        axes[0].fill_between(x, y, alpha=0.3, color='#1976D2')
        axes[0].plot(x, y, color='#1976D2', linewidth=2)

        # COVID shading
        axes[0].axvspan(
            pd.Timestamp('2020-03-01'),
            pd.Timestamp('2021-01-01'),
            alpha=0.08,
            color='red',
            label='COVID Wave 1'
        )
        axes[0].axvspan(
            pd.Timestamp('2021-06-01'),
            pd.Timestamp('2022-01-01'),
            alpha=0.08,
            color='orange',
            label='Delta Outbreak'
        )

        axes[0].set_title('FGRS RCE Monthly Cost (MUSD)', fontsize=13, fontweight='bold')
        axes[0].set_ylabel('Monthly Cost (MUSD)')
        axes[0].legend(loc='upper right')

    # ======================
    # LOGI
    # ======================
    logi = df_mc[df_mc['cost_type'] == 'LOGI_RCE'].copy()
    if not logi.empty:
        logi['date'] = pd.to_datetime(
            logi['year'].astype(int).astype(str)
            + '-'
            + logi['month'].astype(int).astype(str).str.zfill(2)
            + '-01'
        )
        logi = logi.sort_values('date')

        x = logi['date'].to_numpy()

        if 'cumulative_amount_musd' in logi.columns:
            y = logi['cumulative_amount_musd'].to_numpy()
            axes[1].fill_between(x, y, alpha=0.3, color='#388E3C')
            axes[1].plot(x, y, color='#388E3C', linewidth=2)
            axes[1].set_ylabel('Cumulative Cost (MUSD)')
        else:
            y = logi['monthly_amount_musd'].to_numpy()
            axes[1].plot(x, y, color='#388E3C', linewidth=2)
            axes[1].set_ylabel('Monthly Cost (MUSD)')

        axes[1].set_title('LOGI RCE Cost (MUSD)', fontsize=13, fontweight='bold')

    plt.tight_layout()
    plt.savefig(
        os.path.join(OUTPUT_DIR, 'viz_03_monthly_costs.png'),
        dpi=150,
        bbox_inches='tight'
    )
    plt.show()
    print("Saved: viz_03_monthly_costs.png")

else:
    print("Skipped: no monthly cost data")


Saved: viz_03_monthly_costs.png


### 4.4 Personnel on Board (POB) Analysis

In [41]:
if 'tb_t_monthly_pob' in all_tables and len(all_tables['tb_t_monthly_pob']) > 0:
    fig, ax = plt.subplots(figsize=(18, 7))

    pob = all_tables['tb_t_monthly_pob'].copy()
    pob['date'] = pd.to_datetime(pob.apply(lambda r: f"{int(r['year'])}-{int(r['month']):02d}-01", axis=1))
    pob = pob.sort_values('date')

    ax.bar(pob['date'], pob['pob_count'], width=25, color='#1976D2', alpha=0.7, label='POB (CSTS+SubCon)')

    # Isolation overlay
    if 'isolation_count' in pob.columns:
        iso = pob[pob['isolation_count'] > 0]
        if len(iso) > 0:
            ax.bar(iso['date'], iso['isolation_count'], width=25, color='#E53935', alpha=0.7, label='Isolation Facility')

    ax.axhline(y=12000, color='gray', linestyle='--', alpha=0.5, label='Pre-COVID Peak (~12,000)')
    ax.axhline(y=6300, color='orange', linestyle='--', alpha=0.5, label='Restricted Level (~6,300)')

    # COVID shading
    ax.axvspan(pd.Timestamp('2020-03-01'), pd.Timestamp('2021-01-01'), alpha=0.06, color='red')
    ax.axvspan(pd.Timestamp('2021-06-01'), pd.Timestamp('2022-01-01'), alpha=0.06, color='orange')

    ax.set_title('Monthly Personnel on Board (POB) with COVID Impact', fontsize=14, fontweight='bold')
    ax.set_ylabel('Personnel Count')
    ax.legend(loc='upper right')

    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'viz_04_pob_timeline.png'), dpi=150, bbox_inches='tight')
    plt.show()
    print("Saved: viz_04_pob_timeline.png")
else:
    print("Skipped: no POB data")

Saved: viz_04_pob_timeline.png


### 4.5 Subcontractor Performance

In [43]:
if 'tb_t_project_progress' in all_tables and len(all_tables['tb_t_project_progress']) > 0:
    df_prog = all_tables['tb_t_project_progress']

    subcontractors = df_prog['subcontractor'].dropna().unique()
    n_sub = len(subcontractors)

    if n_sub > 0:
        fig, axes = plt.subplots(1, n_sub, figsize=(7 * n_sub, 7), sharey=True)

        if n_sub == 1:
            axes = [axes]

        for idx, (subcon, ax) in enumerate(zip(subcontractors, axes)):
            prog = df_prog[df_prog['subcontractor'] == subcon].copy()

            # Build date safely & fast
            prog['date'] = pd.to_datetime(
                prog['year'].astype(int).astype(str)
                + '-'
                + prog['month'].astype(int).astype(str).str.zfill(2)
                + '-01'
            )
            prog = prog.sort_values('date')

            plan_col = 'plan_progress_pct' if 'plan_progress_pct' in prog.columns else None
            actual_col = 'overall_progress_pct' if 'overall_progress_pct' in prog.columns else None

            # PLAN
            if plan_col:
                plan_mask = prog[plan_col].notna()
                if plan_mask.any():
                    x_plan = prog.loc[plan_mask, 'date'].to_numpy()
                    y_plan = (prog.loc[plan_mask, plan_col] * 100).to_numpy()
                    ax.plot(x_plan, y_plan, linestyle='--', linewidth=2, label='Plan')

            # ACTUAL
            if actual_col:
                actual_mask = prog[actual_col].notna()
                if actual_mask.any():
                    x_actual = prog.loc[actual_mask, 'date'].to_numpy()
                    y_actual = (prog.loc[actual_mask, actual_col] * 100).to_numpy()
                    ax.plot(x_actual, y_actual, linewidth=2, label='Actual')

            ax.set_title(f'{subcon} - Cumulative Progress', fontsize=12, fontweight='bold')

            if idx == 0:
                ax.set_ylabel('Progress (%)')

            ax.set_ylim(0, 105)
            ax.legend()
            ax.tick_params(axis='x', rotation=45)

        plt.suptitle(
            'Subcontractor Progress S-Curves (Plan vs Actual)',
            fontsize=14,
            fontweight='bold',
            y=1.02
        )
        plt.tight_layout()
        plt.savefig(
            os.path.join(OUTPUT_DIR, 'viz_05_subcontractor_progress.png'),
            dpi=150,
            bbox_inches='tight'
        )
        plt.show()
        print("Saved: viz_05_subcontractor_progress.png")

    else:
        print("No subcontractor data to plot")
else:
    print("Skipped: no project progress data")

Saved: viz_05_subcontractor_progress.png


### 4.6 PAMF Claims Deep Dive

In [44]:
if 'tb_t_pamf_claim' in all_tables and len(all_tables['tb_t_pamf_claim']) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(18, 8))

    df_pamf = all_tables['tb_t_pamf_claim']

    # Top 15 sub-categories by claim amount (level 2 or lowest available)
    leaf_level = df_pamf['level'].max()
    top_claims = df_pamf[df_pamf['level'] == leaf_level].nlargest(15, 'claim_amount_usd')

    if len(top_claims) > 0:
        disc_colors = {'COVID': '#E53935', 'LOGISTIC': '#1E88E5', 'PMT': '#43A047', 'SMT': '#FB8C00', 'OTHER': '#757575'}
        bar_colors = [disc_colors.get(g, '#757575') for g in top_claims['pamf_group']]

        axes[0].barh(range(len(top_claims)), top_claims['claim_amount_usd'] / 1e6, color=bar_colors)
        axes[0].set_yticks(range(len(top_claims)))
        axes[0].set_yticklabels(top_claims['label'].str[:40], fontsize=8)
        axes[0].set_xlabel('Claim Amount (Million USD)')
        axes[0].set_title('Top 15 PAMF Claim Categories', fontsize=13, fontweight='bold')
        axes[0].invert_yaxis()

    # Count vs Amount scatter
    disc_data = df_pamf[df_pamf['level'] == 0].copy()
    if len(disc_data) > 0:
        scatter_colors = [disc_colors.get(g, '#757575') for g in disc_data['pamf_group']]
        axes[1].scatter(disc_data['pamf_count'], disc_data['claim_amount_usd'] / 1e6,
                       s=200, alpha=0.7, c=scatter_colors, edgecolors='black', linewidth=0.5)
        for _, row in disc_data.iterrows():
            axes[1].annotate(row['label'], (row['pamf_count'], row['claim_amount_usd'] / 1e6),
                           fontsize=10, fontweight='bold', ha='left',
                           xytext=(5, 5), textcoords='offset points')
        axes[1].set_xlabel('Number of Claims')
        axes[1].set_ylabel('Total Claim Amount (Million USD)')
        axes[1].set_title('PAMF: Claim Count vs Amount', fontsize=13, fontweight='bold')

    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'viz_06_pamf_analysis.png'), dpi=150, bbox_inches='tight')
    plt.show()
    print("Saved: viz_06_pamf_analysis.png")
else:
    print("Skipped: no PAMF data")

Saved: viz_06_pamf_analysis.png


### 4.7 Statistical Summary

In [45]:
print("=" * 70)
print("STATISTICAL SUMMARIES")
print("=" * 70)

if 'tb_t_monthly_cost' in all_tables and len(all_tables['tb_t_monthly_cost']) > 0:
    print("\n=== Monthly Cost Statistics (MUSD) ===")
    print(all_tables['tb_t_monthly_cost'].groupby('cost_type')['monthly_amount_musd'].describe().round(3))

if 'tb_t_monthly_pob' in all_tables and len(all_tables['tb_t_monthly_pob']) > 0:
    print("\n=== Monthly POB Statistics ===")
    print(all_tables['tb_t_monthly_pob'][['pob_count', 'isolation_count']].describe().round(0))

if 'tb_t_pamf_claim' in all_tables and len(all_tables['tb_t_pamf_claim']) > 0:
    print("\n=== PAMF Claim Summary by Group ===")
    disc = all_tables['tb_t_pamf_claim'][all_tables['tb_t_pamf_claim']['level'] == 0].groupby('pamf_group').agg(
        total_claims=('pamf_count', 'sum'),
        total_amount_usd=('claim_amount_usd', 'sum'),
    ).round(2)
    disc['avg_per_claim'] = (disc['total_amount_usd'] / disc['total_claims']).round(2)
    print(disc)

if 'tb_m_amendment' in all_tables:
    print("\n=== Amendment Value Growth ===")
    df_amd = all_tables['tb_m_amendment']
    orig = df_amd.iloc[0]['total_contract_value']
    for _, row in df_amd.iterrows():
        growth = ((row['total_contract_value'] - orig) / orig * 100) if orig else 0
        print(f"  {row['amendment_code']}: ${row['total_contract_value']:>15,.0f}  (+{growth:.1f}%)")

STATISTICAL SUMMARIES

=== Monthly Cost Statistics (MUSD) ===
           count     mean      std     min      25%      50%      75%      max
cost_type                                                                     
FGRS_RCE    56.0  301.189  108.099  15.506  255.882  322.886  382.665  438.904
LOGI_RCE    56.0    7.838    6.444   2.926    4.017    4.534    8.233   24.012

=== Monthly POB Statistics ===
       pob_count  isolation_count
count       54.0             54.0
mean         0.0             66.0
std          0.0            197.0
min          0.0              0.0
25%          0.0              0.0
50%          0.0              0.0
75%          0.0              0.0
max          0.0           1170.0

=== PAMF Claim Summary by Group ===
            total_claims  total_amount_usd  avg_per_claim
pamf_group                                               
COVID                 45      9.011710e+07     2002602.16
LOGISTIC             342      1.569896e+08      459033.94
PMT            

---
## 5. SAVE CLEANED CSVs FOR ALL TABLES

In [46]:
print("Saving cleaned CSVs...\n")
for table_name, df in all_tables.items():
    filepath = os.path.join(OUTPUT_DIR, f'{table_name}.csv')
    df.to_csv(filepath, index=False)
    print(f"  {table_name}.csv  ({len(df)} rows, {len(df.columns)} cols)")

print(f"\nAll {len(all_tables)} table CSVs saved to: {OUTPUT_DIR}")

Saving cleaned CSVs...

  tb_m_project.csv  (1 rows, 11 cols)
  tb_m_amendment.csv  (6 rows, 11 cols)
  tb_m_cost_category.csv  (42 rows, 6 cols)
  tb_m_cost_discipline.csv  (15 rows, 4 cols)
  tb_m_subcontractor.csv  (3 rows, 8 cols)
  tb_m_event.csv  (10 rows, 8 cols)
  tb_t_contract_value.csv  (258 rows, 6 cols)
  tb_t_monthly_cost.csv  (112 rows, 7 cols)
  tb_t_monthly_pob.csv  (54 rows, 7 cols)
  tb_t_pamf_claim.csv  (94 rows, 9 cols)
  tb_t_variation_order.csv  (12 rows, 8 cols)
  tb_t_subcontractor_monthly.csv  (1172 rows, 9 cols)
  tb_t_project_progress.csv  (162 rows, 8 cols)

All 13 table CSVs saved to: D:\BP\data_cleansing


---
## 6. MySQL CREATE TABLE + INSERT SCRIPTS
### Generate DDL and DML for DBeaver import

In [47]:
def get_mysql_type(col_name, dtype, df):
    """Determine MySQL column type from pandas column."""
    col_lower = col_name.lower()
    if col_lower.endswith('_id') and col_lower == df.columns[0].lower():
        return 'INT AUTO_INCREMENT'
    elif col_lower.endswith('_id'):
        return 'INT'
    elif 'date' in col_lower:
        return 'DATE'
    elif 'amount' in col_lower or 'value' in col_lower or 'usd' in col_lower:
        return 'DECIMAL(18,2)'
    elif 'progress' in col_lower or 'pct' in col_lower:
        return 'DECIMAL(10,6)'
    elif dtype in ('float64', 'float32'):
        return 'DECIMAL(18,6)'
    elif dtype in ('int64', 'int32'):
        return 'INT'
    elif col_lower in ('description', 'scope_of_work', 'remarks', 'scope'):
        return 'TEXT'
    else:
        if len(df) > 0:
            max_len = df[col_name].astype(str).str.len().max()
            return f'VARCHAR({max(int(max_len * 1.5), 50)})'
        return 'VARCHAR(255)'

def generate_create_table(table_name, df):
    """Generate MySQL CREATE TABLE statement."""
    lines = [f"DROP TABLE IF EXISTS `{table_name}`;"]
    lines.append(f"CREATE TABLE `{table_name}` (")
    pk_col = df.columns[0]
    col_defs = []
    for col in df.columns:
        mysql_type = get_mysql_type(col, str(df[col].dtype), df)
        nullable = '' if col == pk_col else ' NULL'
        col_defs.append(f"  `{col}` {mysql_type}{nullable}")
    col_defs.append(f"  PRIMARY KEY (`{pk_col}`)")
    lines.append(',\n'.join(col_defs))
    lines.append(") ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;")
    return '\n'.join(lines)

def generate_inserts(table_name, df, batch_size=50):
    """Generate MySQL INSERT statements with batching."""
    if len(df) == 0:
        return f"-- No data for {table_name}"

    statements = []
    cols = ', '.join([f'`{c}`' for c in df.columns])

    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i+batch_size]
        values_list = []
        for _, row in batch.iterrows():
            vals = []
            for v in row.values:
                if v is None or (isinstance(v, float) and np.isnan(v)):
                    vals.append('NULL')
                elif isinstance(v, str):
                    escaped = v.replace("\\", "\\\\").replace("'", "\\'")
                    vals.append(f"'{escaped}'")
                elif isinstance(v, (pd.Timestamp, datetime)):
                    vals.append(f"'{v.strftime('%Y-%m-%d')}'")
                else:
                    vals.append(str(v))
            values_list.append(f"  ({', '.join(vals)})")

        stmt = f"INSERT INTO `{table_name}` ({cols}) VALUES\n" + ',\n'.join(values_list) + ';'
        statements.append(stmt)

    return '\n\n'.join(statements)

print("SQL generator functions defined.")

SQL generator functions defined.


In [48]:
# Generate complete SQL script
all_sql = []
all_sql.append("-- ============================================================")
all_sql.append("-- TEP Data Pipeline - MySQL DDL + DML Script")
all_sql.append(f"-- Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
all_sql.append(f"-- Tables: {len(all_tables)}")
all_sql.append("-- ============================================================")
all_sql.append("")
all_sql.append("SET FOREIGN_KEY_CHECKS = 0;")
all_sql.append("")

# DDL: CREATE TABLE statements
all_sql.append("-- ============================================================")
all_sql.append("-- DDL: CREATE TABLE STATEMENTS")
all_sql.append("-- ============================================================")
all_sql.append("")

# Order: master tables first, then transaction tables
table_order = [
    'tb_m_project', 'tb_m_amendment', 'tb_m_cost_category', 'tb_m_cost_discipline',
    'tb_m_subcontractor', 'tb_m_event',
    'tb_t_contract_value', 'tb_t_monthly_cost', 'tb_t_monthly_pob',
    'tb_t_pamf_claim', 'tb_t_variation_order', 'tb_t_subcontractor_monthly',
    'tb_t_project_progress'
]

for tname in table_order:
    if tname in all_tables:
        ddl = generate_create_table(tname, all_tables[tname])
        all_sql.append(ddl)
        all_sql.append("")

# DML: INSERT statements
all_sql.append("-- ============================================================")
all_sql.append("-- DML: INSERT STATEMENTS")
all_sql.append("-- ============================================================")
all_sql.append("")

for tname in table_order:
    if tname in all_tables:
        all_sql.append(f"-- {tname} ({len(all_tables[tname])} rows)")
        all_sql.append(generate_inserts(tname, all_tables[tname]))
        all_sql.append("")

all_sql.append("SET FOREIGN_KEY_CHECKS = 1;")
all_sql.append("")
all_sql.append("-- End of script")

# Save
sql_content = '\n'.join(all_sql)
sql_path = os.path.join(OUTPUT_DIR, 'tep_mysql_insert.sql')
with open(sql_path, 'w', encoding='utf-8') as f:
    f.write(sql_content)

print(f"MySQL script saved: {sql_path}")
print(f"Script size: {len(sql_content):,} characters")
print(f"Tables included: {sum(1 for t in table_order if t in all_tables)}")

MySQL script saved: D:\BP\data_cleansing\tep_mysql_insert.sql
Script size: 165,596 characters
Tables included: 13


---
## 7. DASHBOARD & VISUALIZATION RECOMMENDATIONS

### Dashboard 1: Executive Cost Overview
- **KPI Cards:** Original Contract ($2.43B), Current ($5.05B), Growth (+108%), Amendments (5)
- **Waterfall chart:** Incremental cost additions per amendment
- **Stacked area:** EPC Lump Sum vs Reimbursable vs Provisional over time
- **Timeline:** Contract milestones with key events annotated

### Dashboard 2: COVID-19 Impact Dashboard
- **Before/After POB:** 12,000+ vs 6,300 peak restriction
- **COVID provisional escalation:** $0 -> $236M -> $408M -> $420M across AMD-3/4/5
- **Isolation facility occupancy** timeline overlay
- **Schedule delay:** Original completion vs extended timeline
- **Cost by COVID type:** Direct, Other Related, Force Majeure, Tier 4

### Dashboard 3: Monthly Cost Burn Rate (FGRS + LOGI)
- **Dual-axis time series:** FGRS monthly + LOGI cumulative with budget lines
- **Cumulative S-curve** vs budget benchmarks per amendment period
- **Monthly burn rate heatmap** (months x years)
- **Budget utilization gauge:** FGRS $1,837M / LOGI $482M targets vs actuals

### Dashboard 4: Subcontractor Performance
- **Triple S-curve:** Meindo vs Penta vs Daewoo (plan vs actual)
- **POB efficiency:** Achieved manhours per person per month
- **Earned Value:** Invoice progression vs % complete
- **Schedule/Cost variance** monthly trend (SV, CV indicators)

### Dashboard 5: PAMF Claims Analysis
- **Treemap:** Claims by discipline > category > sub-category (area = amount)
- **Pareto chart:** Top 20 categories (80/20 rule visualization)
- **SMT dominance:** $1.46B = 82% of total claims breakdown
- **Scatter:** Claim count vs amount with outlier detection

### Dashboard 6: Reimbursable Cost Growth Tracker
- **Amendment-over-amendment growth:** $0 -> $438M -> $1.96B
- **Discipline breakdown:** Construction ($1.09B), Logistics ($447M), PM ($213M)
- **Post-FGRS tracking:** $188M -> $1.51B progression

**Recommended Tools:** Power BI, Tableau, or Plotly Dash for interactive filtering by amendment period, date range, subcontractor, and cost category.

---
## 8. SUMMARY

### Output Files Generated

| File | Type | Description |
|------|------|-------------|
| `tb_m_project.csv` | Master | Project metadata |
| `tb_m_amendment.csv` | Master | 6 contract amendments |
| `tb_m_cost_category.csv` | Master | Hierarchical cost categories |
| `tb_m_cost_discipline.csv` | Master | PAMF discipline taxonomy |
| `tb_m_subcontractor.csv` | Master | 3 subcontractors |
| `tb_m_event.csv` | Master | Key project events |
| `tb_t_contract_value.csv` | Transaction | Contract values per amendment per line item |
| `tb_t_monthly_cost.csv` | Transaction | FGRS + LOGI monthly costs |
| `tb_t_monthly_pob.csv` | Transaction | Monthly POB + isolation counts |
| `tb_t_pamf_claim.csv` | Transaction | PAMF claim details |
| `tb_t_variation_order.csv` | Transaction | Variation orders |
| `tb_t_subcontractor_monthly.csv` | Transaction | Subcontractor monthly metrics |
| `tb_t_project_progress.csv` | Transaction | Plan vs actual progress |
| `tep_mysql_insert.sql` | SQL | Complete CREATE TABLE + INSERT statements for DBeaver |
| `raw_*.csv` | Raw | Raw extraction CSVs |
| `viz_*.png` | Charts | EDA visualization images |

In [49]:
# Final summary
print("=" * 70)
print("TEP DATA PIPELINE - EXECUTION COMPLETE")
print("=" * 70)
print(f"\nOutput directory: {OUTPUT_DIR}")
print(f"\nTables generated:")
for name, df in all_tables.items():
    print(f"  {name}: {len(df)} rows x {len(df.columns)} cols")
print(f"\nSQL script: tep_mysql_insert.sql")
print(f"\nVisualization files: viz_01 through viz_06")

# List all output files
import glob
output_files = glob.glob(os.path.join(OUTPUT_DIR, '*'))
print(f"\nAll output files ({len(output_files)}):")
for f in sorted(output_files):
    size = os.path.getsize(f)
    print(f"  {os.path.basename(f)}: {size:,} bytes")

TEP DATA PIPELINE - EXECUTION COMPLETE

Output directory: D:\BP\data_cleansing

Tables generated:
  tb_m_project: 1 rows x 11 cols
  tb_m_amendment: 6 rows x 11 cols
  tb_m_cost_category: 42 rows x 6 cols
  tb_m_cost_discipline: 15 rows x 4 cols
  tb_m_subcontractor: 3 rows x 8 cols
  tb_m_event: 10 rows x 8 cols
  tb_t_contract_value: 258 rows x 6 cols
  tb_t_monthly_cost: 112 rows x 7 cols
  tb_t_monthly_pob: 54 rows x 7 cols
  tb_t_pamf_claim: 94 rows x 9 cols
  tb_t_variation_order: 12 rows x 8 cols
  tb_t_subcontractor_monthly: 1172 rows x 9 cols
  tb_t_project_progress: 162 rows x 8 cols

SQL script: tep_mysql_insert.sql

Visualization files: viz_01 through viz_06

All output files (28):
  EDA.ipynb: 173,470 bytes
  build_notebook.py: 74,198 bytes
  raw_contract_evolution.csv: 3,251 bytes
  raw_fgrs_monthly.csv: 2,044 bytes
  raw_logi_monthly.csv: 3,087 bytes
  raw_pamf_claims.csv: 8,402 bytes
  raw_pob_monthly.csv: 1,599 bytes
  raw_subcontractor_monthly.csv: 76,229 bytes
  tb_m